In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name

'wikipedia'

In [ ]:
# if there is openai key use it for embeddings
from dotenv import load_dotenv
import os
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))


In [ ]:
# Imports
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Step 1: Load web page documents
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# Step 2: Split documents into smaller chunks for embeddings
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

# Step 3: Initialize embedding model
# OllamaEmbeddings supports multiple models like 'llama3' or 'llama2'
# We use 'nomic-embed-text' here because it's lightweight and CPU-friendly
# If you have a GPU or more CPU resources, you could use 'llama3' or 'llama2' for higher-dimensional embeddings
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# Step 4: Create FAISS vector store from documents
vectordb = FAISS.from_documents(documents, embedding_model)

# Step 5: Create a retriever to fetch relevant chunks for queries
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023F8387A840>, search_kwargs={})

In [8]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith - Docs by LangChain', 'language': 'en'}, page_content='Get started with LangSmith - Docs by LangChainSkip to main contentOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubTry LangSmithGet started with LangSmithCopy pageCopy pageLangSmith is a platform for building product

In [14]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [15]:
retriever_tool.name

'langsmith_search'

In [16]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [17]:
tools=[wiki,arxiv,retriever_tool]

In [18]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\miniconda_setup\\envs\\langchain_env\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000023FC0CC4540>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_com

In [20]:
from langchain_ollama import OllamaLLM

# Initialize Llama 3 model locally
llm = OllamaLLM(model="llama3")


In [21]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
from langchain_ollama import OllamaLLM
from langchain.agents import initialize_agent, Tool, AgentExecutor

In [29]:
from langchain_ollama import OllamaLLM
from langchain.agents import initialize_agent, Tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

# Step 1: LLM
llm = OllamaLLM(model="llama3")  # runs locally

# Step 2: Tool
wiki = WikipediaAPIWrapper()
wikipedia_tool = WikipediaQueryRun(api_wrapper=wiki)
tools = [Tool(name="Wikipedia", func=wikipedia_tool.run, description="Search Wikipedia for facts")]

# Step 3: Initialize agent (no separate AgentExecutor needed)
agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent="zero-shot-react-description",
    verbose=True
)

# Step 4: Run query directly using agent.run()
response = agent.run("Tell me about Langsmith")
print(response)




> Entering new AgentExecutor chain...
A new question! Let's see what Wikipedia has to say.

Thought: It seems like Langsmith is not a well-known topic, so I should try searching for it on Wikipedia.

Action: Wikipedia

Action Input: "Langsmith"

Observation: Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.
Thought:Here's my response:

Question: Tell me about Langsmith
Thought:A new question! Let's see what Wikipedia has to say.

Thought: It seems like Langsmith is not a well-known topic, so I should try searching for it on Wikipedia.

Action: Wikipedia

Action Input: "Langsmith"

Observation: Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 

In [30]:
response = agent.run("Tell me about Python programming language")
print(response)



> Entering new AgentExecutor chain...
Let's get started.

Thought: To answer this question, I'll use Wikipedia to gather information about the Python programming language.

Action: Wikipedia

Action Input: "Python (programming language)"

Observation: Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language. Python 3.0, released in 2008, was a major revision and not completely backward-compatible with earlier versions. Recent versions, such as Python 3.13, 3.12 and older (and 3.14), have added capabilities and keywords for typing (and more; e.g. incre